In [2]:
#Load the picke file
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [3]:
## Load the trained model ,scaler, pickle,one hot
model = load_model('model.h5')


##load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [17]:
# Example input data for a single prediction (as seen in image_431420.png)
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [18]:
input_data['Gender'] = label_encoder_gender.transform([input_data['Gender']])[0]

In [19]:
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 1,
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [25]:
# Correct input shape for OneHotEncoder
geo_value = [[input_data['Geography']]]

# Transform (do NOT use .toarray() because your encoder returns dense data)
geo_encoded = label_encoder_geo.transform(geo_value)

# Convert to DataFrame
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=label_encoder_geo.get_feature_names_out(['Geography'])
)


C:\Users\Yash Gehlot\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [27]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [50]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [51]:
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [52]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [43]:
#Scaling input data
input_df=scaler.transform(input_df)

In [46]:
#Predict 
prediction=model.predict(input_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [47]:
prediction_proba = prediction[0][0]

In [48]:
prediction_proba

0.016628047

In [49]:
if prediction_proba>0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
